# P2 Perceptron Learning Rule

Om de perceptron learning rule te implementeren moeten we de volgende begrippen implementeren:

- `target`, `input`, `weights` en `bias`.
- `output = activation(weighted_sum)`.
- `error = target - output`.
- `eta = 0.1`, wordt ook wel de learning rate genoemd en is een hyperparameter.
- `delta_weight = eta * error * input`.
- `delta_bias = eta * error`.
- `weight_derivative = weight + delta_weight`.
- `bias_derivative = bias + delta_bias`.
- `mean_squared_error = sum(error**2) / n`.